In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Apr 24 15:17:47 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Reinforcement learning using DQN and CNN

### Improvement Ideas

- Make the network deeper
- Normalize the Volume as well
- Make use of the extra features I have in the data
- Use LSTM model instead of CNN
    - That would require some rewriting of the gym so that it only feeds in one timestep observation at a time
- Train the network on multiple different crypto's and see if that helps it generalize for data it hasn't seen before

## Training the Model

In [2]:
!pip3 install ptan --quiet

     |████████████████████████████████| 773.1MB 23kB/s 
ERROR: torchvision 0.5.0 has requirement torch==1.4.0, but you'll have torch 1.3.0 which is incompatible.


In [0]:
BASE_PATH = '/content/drive/My Drive/NTNU/6. Semester/NEURO140'

In [0]:
%matplotlib notebook
import os
import gym
from gym import wrappers
import ptan
import argparse
import numpy as np
import pandas as pd
from datetime import datetime

import torch
import torch.optim as optim

os.chdir(os.path.join(BASE_PATH, 'agent1'))

from lib import environ, models, data, common, validation

from torch.utils.tensorboard import SummaryWriter

In [0]:
BATCH_SIZE = 32

BARS_COUNT = 50
RESET_ON_CLOSE = True
RANDOM_OFS_ON_RESET = True
REWARD_ON_CLOSE = True
STATE_1D = True
VOLUMES = True

TARGET_NET_SYNC = 1000

# INTERVAL = '1M'
# PAIR = 'YNDX'
# DATA_FILENAME = 'YNDX_150101_151231.csv'
# DATA_PATH = os.path.join(BASE_PATH, "Data", INTERVAL, PAIR, DATA_FILENAME)
data_paths = {
    'BTCUSD': '/content/drive/My Drive/NTNU/6. Semester/NEURO140/Data/1M/BTCUSD/COINBASE_BTCUSD, 1.csv',
    'BTCEUR': '/content/drive/My Drive/NTNU/6. Semester/NEURO140/Data/1M/BTCEUR/COINBASE_BTCEUR, 1.csv',
    'BCHUSD': '/content/drive/My Drive/NTNU/6. Semester/NEURO140/Data/1M/BCHUSD/KRAKEN_BCHUSD, 1.csv',
    'EOSUSD': '/content/drive/My Drive/NTNU/6. Semester/NEURO140/Data/1M/EOSUSD/KRAKEN_EOSUSD, 1.csv',
    'LTCUSD': '/content/drive/My Drive/NTNU/6. Semester/NEURO140/Data/1M/LTCUSD/KRAKEN_LTCUSD, 1.csv',
    'XMRUSD': '/content/drive/My Drive/NTNU/6. Semester/NEURO140/Data/1M/XMRUSD/KRAKEN_XMRUSD, 1.csv',
    'XRPEUR': '/content/drive/My Drive/NTNU/6. Semester/NEURO140/Data/1M/XRPEUR/KRAKEN_XRPEUR, 1.csv',
    'XRPUSD': '/content/drive/My Drive/NTNU/6. Semester/NEURO140/Data/1M/XRPUSD/KRAKEN_XRPUSD, 1.csv',
    'XTZUSD': '/content/drive/My Drive/NTNU/6. Semester/NEURO140/Data/1M/XTZUSD/KRAKEN_XTZUSD, 1.csv',
}

# VALIDATION_FILENAME = 'YNDX_160101_161231.csv'
# VALIDATION_PATH = os.path.join(BASE_PATH, "Data", INTERVAL, PAIR, VALIDATION_FILENAME)
validation_paths = {
    'ETHEUR': '/content/drive/My Drive/NTNU/6. Semester/NEURO140/Data/1M/ETHEUR/KRAKEN_ETHEUR, 1 (1).csv',
    'ETHUSD': '/content/drive/My Drive/NTNU/6. Semester/NEURO140/Data/1M/ETHUSD/KRAKEN_ETHUSD, 1 (3).csv',
}

GAMMA = 0.99

REPLAY_SIZE = 100_000
REPLAY_INITIAL = 10_000

REWARD_STEPS = 2

LEARNING_RATE = 0.0001

STATES_TO_EVALUATE = 1000
EVAL_EVERY_STEP = 1000

# EPSILON_START = 1.0
EPSILON_START = 1.0
EPSILON_STOP = 0.1
EPSILON_STEPS = 1_000_000

CHECKPOINT_EVERY_STEP = 1_000_000
VALIDATION_EVERY_STEP = 100_000

In [0]:
# Helper function for creating the Prices-tuple
def get_tuple_from_df(df):
    return data.Prices(
        open=np.array(df['open']),
        high=np.array(df['high']),
        low=np.array(df['low']),
        close=np.array(df['close']),
        volume=np.array(df['Volume']),
    )

In [0]:
def prices_to_relative(prices, normalize_volume=False):
    """
    Convert prices to relative in respect to open price
    :param ochl: tuple with open, close, high, low
    :return: tuple with open, rel_close, rel_high, rel_low
    """
    assert isinstance(prices, data.Prices)
    rh = (prices.high - prices.open) / prices.open
    rl = (prices.low - prices.open) / prices.open
    rc = (prices.close - prices.open) / prices.open

    volume = prices.volume
    if normalize_volume:
        vol_cpy = prices.volume.copy()
        vol_cpy[0] = 0

        for i, num in enumerate(volume[:-1], start=1):
            d = volume[i] - num
            if d == 0:
                vol_cpy[i] = 0
            elif num == 0:
                vol_cpy[i] = 1
            else:
                vol_cpy[i] = d / num

        volume = vol_cpy

    return data.Prices(open=prices.open, high=rh, low=rl, close=rc, volume=volume)

In [0]:
# Get training data
stock_data = {}
for pair, path in data_paths.items():
    df = pd.read_csv(path)
    data_tuple = get_tuple_from_df(df)
    stock_data[pair] = prices_to_relative(data_tuple, normalize_volume=True)

val_data = {}
for pair, path in validation_paths.items():
    df = pd.read_csv(path)
    data_tuple = get_tuple_from_df(df)
    val_data[pair] = prices_to_relative(data_tuple, normalize_volume=True)

In [0]:
import torch.nn as nn
import torch.nn.functional as F


class DQNConv1DLarge(nn.Module):
    def __init__(self, shape, actions_n):
        super(DQNConv1DLarge, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv1d(in_channels=shape[0], out_channels=64, kernel_size=3, padding=1),
            nn.MaxPool1d(3, 2),
            nn.ReLU(),
            nn.Dropout2d(p=0.1),
            nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, padding=1),
            nn.MaxPool1d(3, 2),
            nn.ReLU(),
            nn.Dropout2d(p=0.1),
            # nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            # nn.MaxPool1d(3, 2),
            # nn.ReLU(),
            # nn.Dropout2d(p=0.1),
            nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout2d(p=0.1),
            nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout2d(p=0.1),
            nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
        )

        out_size = self._get_conv_out(shape)

        self.fc_val = nn.Sequential(
            nn.Linear(out_size, 512),
            nn.ReLU(),
            nn.Dropout(),
            # nn.Linear(512, 512),
            # nn.ReLU(),
            # nn.Dropout2d(),
            nn.Linear(512, 1)
        )

        self.fc_adv = nn.Sequential(
            nn.Linear(out_size, 512),
            nn.ReLU(),
            nn.Dropout(),
            # nn.Linear(512, 512),
            # nn.ReLU(),
            # nn.Dropout2d(),
            nn.Linear(512, actions_n)
        )

    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        conv_out = self.conv(x).view(x.size()[0], -1)
        val = self.fc_val(conv_out)
        adv = self.fc_adv(conv_out)
        return val + adv - adv.mean(dim=1, keepdim=True)

In [10]:
step_idx = 0
eval_states = None
best_mean_val = None

# device = xm.xla_device()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

run_name = "test-6.4-crypto-agent-small"  # + str(datetime.now())

saves_path = os.path.join(BASE_PATH, "agent1", "crypto-saves", run_name)
print(f"Saving to path:\n{saves_path}")

os.makedirs(saves_path, exist_ok=True)

env = environ.StocksEnv(
    stock_data,
    bars_count=BARS_COUNT,
    reset_on_close=RESET_ON_CLOSE,
    random_ofs_on_reset=RANDOM_OFS_ON_RESET,
    reward_on_close=REWARD_ON_CLOSE,
    state_1d=STATE_1D,
    volumes=VOLUMES,
)

env_tst = environ.StocksEnv(
    stock_data,
    bars_count=BARS_COUNT,
    reset_on_close=RESET_ON_CLOSE,
    random_ofs_on_reset=RANDOM_OFS_ON_RESET,
    reward_on_close=REWARD_ON_CLOSE,
    state_1d=STATE_1D,
    volumes=VOLUMES,
)

env = wrappers.TimeLimit(env, max_episode_steps=1000)

env_val = environ.StocksEnv(
    val_data,
    bars_count=BARS_COUNT,
    reset_on_close=RESET_ON_CLOSE,
    random_ofs_on_reset=RANDOM_OFS_ON_RESET,
    reward_on_close=REWARD_ON_CLOSE,
    state_1d=STATE_1D,
    volumes=VOLUMES,
)

writer = SummaryWriter(comment="-conv-" + run_name + "")
net = models.DQNConv1D(env.observation_space.shape, env.action_space.n).to(device)
# Uncomment next line to use deeper CNN
# net = DQNConv1DLarge(env.observation_space.shape, env.action_space.n).to(device)

print(net)

tgt_net = ptan.agent.TargetNet(net)
selector = ptan.actions.EpsilonGreedyActionSelector(EPSILON_START)
agent = ptan.agent.DQNAgent(net, selector, device=device)
exp_source = ptan.experience.ExperienceSourceFirstLast(
    env, agent, GAMMA, steps_count=REWARD_STEPS
)
buffer = ptan.experience.ExperienceReplayBuffer(exp_source, REPLAY_SIZE)
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)


load_state = '/content/drive/My Drive/NTNU/6. Semester/NEURO140/agent1/crypto-saves/test-6.4-crypto-agent-small/checkpoint-32.data'
if load_state is not None:
    state = torch.load(load_state)
    net.load_state_dict(state['model_state_dict'])
    optimizer.load_state_dict(state['optimizer_state_dict'])
    step_idx = state['step_idx']
    best_mean_val = state['best_mean_val']
    print(f"State loaded –> step index: {step_idx}, best mean val: {best_mean_val}")

    net.train()

Using device: cuda
Saving to path:
/content/drive/My Drive/NTNU/6. Semester/NEURO140/agent1/crypto-saves/test-6.4-crypto-agent-small
DQNConv1D(
  (conv): Sequential(
    (0): Conv1d(6, 128, kernel_size=(5,), stride=(1,))
    (1): ReLU()
    (2): Conv1d(128, 128, kernel_size=(5,), stride=(1,))
    (3): ReLU()
  )
  (fc_val): Sequential(
    (0): Linear(in_features=5376, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=1, bias=True)
  )
  (fc_adv): Sequential(
    (0): Linear(in_features=5376, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=3, bias=True)
  )
)
State loaded –> step index: 32000000, best mean val: 828585.9736392498


In [11]:
# main training loop
with common.RewardTracker(writer, np.inf, group_rewards=100) as reward_tracker:
    while True:
        step_idx += 1
        buffer.populate(1)
        selector.epsilon = max(
            EPSILON_STOP, EPSILON_START - step_idx / EPSILON_STEPS
        )

        new_rewards = exp_source.pop_rewards_steps()
        if new_rewards:
            reward_tracker.reward(new_rewards[0], step_idx, selector.epsilon)

        if len(buffer) < REPLAY_INITIAL:
            continue

        if eval_states is None:
            print("Initial buffer populated, start training")
            eval_states = buffer.sample(STATES_TO_EVALUATE)
            eval_states = [
                np.array(transition.state, copy=False) for transition in eval_states
            ]
            eval_states = np.array(eval_states, copy=False)

        if step_idx % EVAL_EVERY_STEP == 0:
            mean_val = common.calc_values_of_states(eval_states, net, device=device)
            writer.add_scalar("values_mean", mean_val, step_idx)
            if best_mean_val is None or best_mean_val < mean_val:
                if best_mean_val is not None:
                    print(f"{step_idx}: Best mean value updated {best_mean_val:.3f} -> {mean_val:.3f}")
                best_mean_val = mean_val
                torch.save({
                                "model_state_dict": net.state_dict(),
                                "optimizer_state_dict": optimizer.state_dict(),
                                "step_idx": step_idx,
                                "best_mean_val": best_mean_val,
                            },
                            os.path.join(saves_path, f"mean_val-{mean_val:.3f}.data"))
                torch.save(net,
                            os.path.join(saves_path, f"mean_val-{mean_val:.3f}-fullmodel.data"))

        optimizer.zero_grad()
        batch = buffer.sample(BATCH_SIZE)
        loss_v = common.calc_loss(
            batch, net, tgt_net.target_model, GAMMA ** REWARD_STEPS, device=device
        )
        loss_v.backward()
        optimizer.step()

        if step_idx % TARGET_NET_SYNC == 0:
            tgt_net.sync()

        if step_idx % CHECKPOINT_EVERY_STEP == 0:
            idx = step_idx // CHECKPOINT_EVERY_STEP
            torch.save({
                            "model_state_dict": net.state_dict(),
                            "optimizer_state_dict": optimizer.state_dict(),
                            "step_idx": step_idx,
                            "best_mean_val": best_mean_val,
                        },
                os.path.join(saves_path, f"checkpoint-{idx}.data"),
            )
            torch.save(net,
                os.path.join(saves_path, f"fullmodel-{idx}.data"),
            )

        if step_idx % VALIDATION_EVERY_STEP == 0:
            res = validation.validation_run(env_tst, net, device=device)
            for key, val in res.items():
                writer.add_scalar(f"{key.capitalize()}/train", val, step_idx)
            res = validation.validation_run(env_val, net, device=device)
            for key, val in res.items():
                writer.add_scalar(f"{key.capitalize()}/validate", val, step_idx)

print("Training done")

32 003 602: done 100 games, mean reward -0.210, mean steps 36.01, speed 12296800.73 f/s, eps 0.10
32 006 568: done 200 games, mean reward -0.186, mean steps 32.84, speed 1362.48 f/s, eps 0.10
Initial buffer populated, start training
32 010 120: done 300 games, mean reward -0.178, mean steps 33.73, speed 1099.56 f/s, eps 0.10
32 016 893: done 400 games, mean reward -0.162, mean steps 42.23, speed 169.17 f/s, eps 0.10
32 022 618: done 500 games, mean reward -0.113, mean steps 45.23, speed 170.26 f/s, eps 0.10
32 027 278: done 600 games, mean reward -0.156, mean steps 45.46, speed 169.17 f/s, eps 0.10
32 030 615: done 700 games, mean reward -0.149, mean steps 43.73, speed 170.96 f/s, eps 0.10
32 035 753: done 800 games, mean reward -0.187, mean steps 44.69, speed 170.61 f/s, eps 0.10
32 040 037: done 900 games, mean reward -0.223, mean steps 44.48, speed 168.99 f/s, eps 0.10
32 045 359: done 1000 games, mean reward -0.243, mean steps 45.36, speed 170.50 f/s, eps 0.10
32 050 042: done 1100

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DQNConv1D. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv1d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/di

33 001 481: done 20800 games, mean reward -0.231, mean steps 48.57, speed 68.19 f/s, eps 0.10
33 007 118: done 20900 games, mean reward -0.232, mean steps 48.80, speed 164.82 f/s, eps 0.10
33 012 202: done 21000 games, mean reward -0.232, mean steps 48.87, speed 168.03 f/s, eps 0.10
33 017 367: done 21100 games, mean reward -0.231, mean steps 48.90, speed 168.84 f/s, eps 0.10
33 021 470: done 21200 games, mean reward -0.233, mean steps 48.76, speed 169.02 f/s, eps 0.10
33 027 812: done 21300 games, mean reward -0.236, mean steps 48.77, speed 168.06 f/s, eps 0.10
33 031 809: done 21400 games, mean reward -0.234, mean steps 48.82, speed 166.66 f/s, eps 0.10
33 037 349: done 21500 games, mean reward -0.235, mean steps 48.83, speed 168.58 f/s, eps 0.10
33 042 464: done 21600 games, mean reward -0.238, mean steps 48.82, speed 167.88 f/s, eps 0.10
33 047 005: done 21700 games, mean reward -0.237, mean steps 48.65, speed 167.91 f/s, eps 0.10
33 052 017: done 21800 games, mean reward -0.236, m

KeyboardInterrupt: ignored